In [57]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [58]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [59]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

0 experiments


In [60]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

0 experiments before preprocessing


""


In [61]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [62]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

0 experiments after preprocessing


""


In [63]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

ValueError: Cannot set a DataFrame without columns to the column params.qa.technique

In [ ]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [ ]:
# make temperature 1 decimal
df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [45]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

In [46]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [47]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [48]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [49]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.split": ["train"],
    "params.qa.temperature": [
        # 0.0,
        0.5,
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_sc": [
        1,
    ],
    "params.qa.n_shot": [
        0,
        1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [50]:
varying_params_list = [
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["direct/format-few.txt"],
        "params.qa.few_shot_examples": ["manual/direct.json"],
    },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cot/format-thought.txt"],
    #     "params.qa.few_shot_examples": ["manual/cot.json"],
    # },
    {
        "params.qa.model": [
            "llama-3-8b",
            # "llama-3.3-70b",
        ],
        "params.qa.system_prompt": ["ccot/format-thought.txt"],
        "params.qa.few_shot_examples": ["manual/ccot.json"],
    },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cok/adapted.txt"],
    #     "params.qa.few_shot_examples": ["manual/cok.json"],
    # },
    # {
    #     "params.qa.model": [
    #         "llama-3-8b",
    #         # "llama-3.3-70b",
    #     ],
    #     "params.qa.system_prompt": ["cte/format-triples-ere-few.txt"],
    #     "params.qa.few_shot_examples": ["manual/cte-triples.json"],
    # },
    # {
    #     "params.qa.model": [
    #         "deepseek-r1-llama-8b",
    #         "deepseek-r1-llama-70b",
    #     ],
    #     "params.qa.system_prompt": ["empty.txt"],
    #     "params.qa.user_prompt_template": ["icq-format.txt"],
    #     "params.qa.few_shot_examples": ["manual/reasoning.json"],
    # },
]

In [51]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

12 experiment configurations
['params.dataset.path', 'params.dataset.split', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [52]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 0


In [53]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

12 missing configurations


In [54]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [55]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [56]:
df

""
